In [28]:
from nltk import Tree
import networkx as nx
import re
import spacy


nlp = spacy.load("zh_core_web_trf")
# nlp = spacy.load("zh_core_web_sm")
# nlp = spacy.load("en_core_web_sm")

In [45]:
doc = nlp("中國晉元帝在建康稱晉王")
entity1 = '元帝'
entity2 = '王'

print('sentence:'.format(doc))
# Load spacy's dependency tree into a networkx graph
edges = []
edges_look_up = []
edges_and_dependencies_look_up = []
edges_and_dependencies = []

for token in doc:
    for child in token.children:
        edges_look_up.append([token.lower_,
                             child.lower_])
        edges_and_dependencies_look_up.append([(token.lower_, token.dep_),
                                             (child.lower_, child.dep_) ])
        
        edges.append(('{0}'.format(token.lower_),
                      '{0}'.format(child.lower_)))
        edges_and_dependencies.append(('{0}'.format((token.lower_, token.dep_)),
                      '{0}'.format((child.lower_, child.dep_))))
        
graph = nx.Graph(edges)
graph_test = nx.Graph(edges_and_dependencies)
# Get the length and path





first_entity_index_list = find_element_nested_list(entity1, edges_look_up, [])
second_entity_index_list = find_element_nested_list(entity2, edges_look_up, [])
e_1 = edges_and_dependencies[int(first_entity_index_list[0].split("-")[0])][int(first_entity_index_list[0].split("-")[1])]
e_2 = edges_and_dependencies[int(second_entity_index_list[0].split("-")[0])][int(second_entity_index_list[0].split("-")[1])]

print(e_1, e_2)

shortest_path_list = nx.shortest_path(graph_test, source=e_1, target=e_2)

for elementIndex, element in enumerate(shortest_path_list):
    if elementIndex not in [0, len(shortest_path_list) - 1]:
        print(re.split("'", element)[3])
    else:
        print(re.split("'", element)[1])



sentence:
('元帝', 'nsubj') ('王', 'dobj')
元帝
ROOT
王
